#沸火工作室修改 ----低清转换高清
##原作   esrgan
##本版本默认弄到google drive
###把图片/视频放到lr文件夹就ok了

In [ ]:
!nvidia-smi #recommended gpus are p100 and 
!git clone https://github.com/wikejohn/ESRGAN
!rm -f /content/ESRGAN/LR/0
from google.colab import drive
import gdown
drive.mount('/content/gdrive')



print("Downloading pretrained models")
output1 = '/content/ESRGAN/models/RRDB_ESRGAN_x4.pth'
output2 = '/content/ESRGAN/models/RRDB_PSNR_x4.pth'
output3 = '/content/ESRGAN/models/PPON_D.pth'
output4 = '/content/ESRGAN/models/PPON_G.pth'
print ('Downloading RRDB_ESRGAN_x4.pth')
gdown.download('https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene', output1, quiet=True) 
print ('Downloading RRDB_PSNR_x4.pth')
gdown.download('https://drive.google.com/uc?id=1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN', output2, quiet=True)
print ('Downloading PPON_D.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=1Fr5aKCD6mw6P-hI0BZr6My2gHNhtUk-V', output3, quiet=True) 
print ('Downloading PPON_G.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=12uR3BSftNA0HDYiKda23GyAj_crpSjOm', output4, quiet=True)  

Mon Dec 13 09:23:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    57W / 149W |   9479MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

'/content/ESRGAN/models/PPON_G.pth'

上传视频的话要拆分，用这个


In [ ]:
%cd /content/ESRGAN/LR
!ffmpeg  -i /content/ESRGAN/video_input/* %04d.png

/content/ESRGAN/LR
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --e

前面完事运行这个


In [ ]:
#@title Upscale images/video frames
%cd /content/ESRGAN
from datetime import datetime
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests
import imageio
import requests
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
from google.colab import files

Choose_device = "cuda"  #@param ["cuda","cpu"]

model_path = 'models/RRDB_PSNR_x4.pth' #@param ['models/RRDB_ESRGAN_x4.pth','models/RRDB_PSNR_x4.pth','models/PPON_G.pth','models/PPON_D.pth']  
device = torch.device(Choose_device) 


test_img_folder = 'LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
a=0
for path in glob.glob(test_img_folder):
    a=a+1
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    
    dt01 = datetime.today()
    fs=str(dt01.date()) + "(" +str(a) + ")" 
    cv2.imwrite('/content/gdrive/MyDrive/fchqphotos/{:s}.png'.format(fs), output)

/content/ESRGAN
Model path models/RRDB_PSNR_x4.pth. 
Testing...
1 a0mgx-rm5zx
2 abhfe-a47zc
3 abtza-j3413


如果是视频的话要打包成这个

In [ ]:
!ffmpeg -f image2 -framerate 25 -i /content/ESRGAN/results/%04d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p output.mp4

打包成zip

In [ ]:
from google.colab import files

!zip -r results.zip /content/ESRGAN/results

  adding: content/ESRGAN/results/ (stored 0%)
  adding: content/ESRGAN/results/comic_rlt.png (deflated 0%)
  adding: content/ESRGAN/results/.ipynb_checkpoints/ (stored 0%)
  adding: content/ESRGAN/results/k2.png (deflated 1%)
  adding: content/ESRGAN/results/baboon.png (deflated 0%)
  adding: content/ESRGAN/results/baboon_rlt.png (deflated 0%)
  adding: content/ESRGAN/results/comic.png (deflated 0%)


下载zip

In [ ]:
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

#move zip file to google drive
shutil.move("./results.zip", "/content/gdrive/My Drive/results.zip")